In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/undersampleddatausingrepeatedenn/undersampled_using_repeated_enn.csv


In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('/kaggle/input/undersampleddatausingrepeatedenn/undersampled_using_repeated_enn.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,5184,754,6,74,1,2,0.0,0.0,0.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,171951,738,17,3003325,4,0,2064.0,0.0,516.0,516.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TFTP
2,57976,2279,17,257,32,0,13936.0,0.0,440.0,368.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_NTP
3,33152,1009,17,3,2,0,826.0,0.0,413.0,413.0,...,402,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_MSSQL
4,4518,2648,6,589,4,0,123.0,0.0,46.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


Simple Bi-GRU Implementation

In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Load the dataset
#df = pd.read_csv('data.csv')

# Split the data into features (X) and label (y)
X = df.drop('Label', axis=1).values
y = df['Label'].values

features = df.drop("Label", axis=1).values
labels = df["Label"].values

# One-hot encode the label
#y = tf.keras.utils.to_categorical(y)

encoder = LabelEncoder()
y = encoder.fit_transform(y)
y = to_categorical(y)

# Split the data into training and testing sets
train_X = X[:int(X.shape[0] * 0.8), :]
train_y = y[:int(y.shape[0] * 0.8), :]

test_X = X[int(X.shape[0] * 0.8):, :]
test_y = y[int(y.shape[0] * 0.8):, :]

# Define the model
model = tf.keras.Sequential()

# Add the first GRU layer
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(units=64, return_sequences=True), input_shape=(features.shape[1], 1)))

# Add a dropout layer to prevent overfitting
model.add(tf.keras.layers.Dropout(0.5))

# Add the second GRU layer
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(units=32)))

# Add a dropout layer to prevent overfitting
model.add(tf.keras.layers.Dropout(0.5))

# Add a dense layer for the output
model.add(tf.keras.layers.Dense(units=train_y.shape[1], activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(train_X, train_y, batch_size=32, epochs=5, validation_split=0.2)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_X, test_y)

# Print the test accuracy
print("Test Accuracy: {}".format(test_accuracy))
print("Test Loss: {}".format(test_loss))


Epoch 1/5
4440/4440 [==============================] - 428s 95ms/step - loss: 0.4409 - accuracy: 0.8500 - val_loss: 0.2289 - val_accuracy: 0.9205
Epoch 2/5
4440/4440 [==============================] - 421s 95ms/step - loss: 0.2674 - accuracy: 0.9055 - val_loss: 0.2004 - val_accuracy: 0.9241
Epoch 3/5
4440/4440 [==============================] - 419s 94ms/step - loss: 0.2388 - accuracy: 0.9131 - val_loss: 0.2022 - val_accuracy: 0.9279
Epoch 4/5
4440/4440 [==============================] - 420s 95ms/step - loss: 0.2263 - accuracy: 0.9163 - val_loss: 0.1839 - val_accuracy: 0.9318
Epoch 5/5
1388/1388 [==============================] - 29s 21ms/step - loss: 0.1731 - accuracy: 0.9366
Test Accuracy: 0.9365919828414917
Test Loss: 0.173060342669487


Using optuna


In [ ]:
import optuna
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Load the dataset
#df = pd.read_csv('data.csv')

# Split the data into features (X) and label (y)
X = df.drop('Label', axis=1).values
y = df['Label'].values

features = df.drop("Label", axis=1).values
labels = df["Label"].values

# One-hot encode the label
#y = tf.keras.utils.to_categorical(y)

encoder = LabelEncoder()
y = encoder.fit_transform(y)
y = to_categorical(y)

# Split the data into training and testing sets
train_X = X[:int(X.shape[0] * 0.8), :]
train_y = y[:int(y.shape[0] * 0.8), :]

test_X = X[int(X.shape[0] * 0.8):, :]
test_y = y[int(y.shape[0] * 0.8):, :]

# Define the objective function to be minimized
def objective(trial):
    # Define the model
    model = tf.keras.Sequential()

    # Add the first GRU layer
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(units=trial.suggest_int("units_1", 16, 128), return_sequences=True), input_shape=(features.shape[1], 1)))

    # Add a dropout layer to prevent overfitting
    model.add(tf.keras.layers.Dropout(trial.suggest_float("dropout_1", 0, 0.5)))

    # Add the second GRU layer
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(units=trial.suggest_int("units_2", 16, 128))))

    # Add a dropout layer to prevent overfitting
    model.add(tf.keras.layers.Dropout(trial.suggest_float("dropout_2", 0, 0.5)))

    # Add a dense layer for the output
    model.add(tf.keras.layers.Dense(units=train_y.shape[1], activation='softmax'))

    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Train the model
    history = model.fit(train_X, train_y, batch_size=32, epochs=5, validation_split=0.2)

    # Evaluate the model on the test set
    test_loss, test_accuracy = model.evaluate(test_X, test_y)

    # Return the test loss as the objective value to be minimized
    return test_loss

# Create a study and optimize the objective function
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)

# Print the best parameters and objective value
best_params = study.best_params
best_value = study.best_value
print("Best Parameters: {}".format(best_params))
print("Best Value: {}".format(best_value))


[I 2023-02-19 20:52:38,902] A new study created in memory with name: no-name-a56e621b-ecd2-487d-886a-4efae1f87d6e


Epoch 1/5
4440/4440 [==============================] - 541s 120ms/step - loss: 0.3108 - accuracy: 0.8871 - val_loss: 0.1989 - val_accuracy: 0.9274
Epoch 2/5
4440/4440 [==============================] - 534s 120ms/step - loss: 0.2115 - accuracy: 0.9194 - val_loss: 0.2652 - val_accuracy: 0.9055
Epoch 3/5
4440/4440 [==============================] - 533s 120ms/step - loss: 0.1989 - accuracy: 0.9232 - val_loss: 0.1715 - val_accuracy: 0.9351
Epoch 4/5
 876/4440 [====>.........................] - ETA: 6:46 - loss: 0.1881 - accuracy: 0.9277